# 4. Data Transformation - 연봉 데이터 다루기
## 4-1. 학습목표: 데이터 변환의 고급 기술을 습득합니다.
1. 데이터를 다양한 방법으로 합치고 변환할 수 있다.
2. 데이터의 스케일을 변환할 수 있다.
3. 카테고리형 데이터를 숫자형태로 변환할 수 있다.
4. 데이터의 차원축소를 할 수 있다.

## 4-2. Data merge(데이터 병합)
1. merge와 join의 특징에 대해 더 알아 봅시다.
2. pandas에서는 merge 함수를 사용하여 SQL 스타일의 조인을 수행할 수 있습니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 병합 실습용 데이터프레임 생성

df_a = pd.DataFrame({'key': ['a', 'b', 'c', 'd', 'e'], 'num_a': [1, 2, 3, 4, 5]})
df_b = pd.DataFrame({'key': ['a', 'b', 'd', 'f', 'g'], 'num_b': [11, 15, 35, 45, 55]})
df_c = pd.DataFrame({'key': ['f', 'g', 'h', 'i', 'j'], 'num_a': [6, 7, 8, 9, 0]})

### 4-2-1.concat() 함수
- pd.concat(axis = int) = 데이터를 int 방향으로 결합; axis 생략시 axis = 0
- 병합할 데이터프레임에 서로 다른 컬럼명이 있을 경우, 컬럼 추가
- 결합방향의 인덱스나 컬럼은 재정렬되지 않고 덧붙여짐에 주목

In [3]:
# axis = 0 혹은 생략의 경우 인덱스는 재정렬되지 않고 이어붙여짐에 주목!
pd.concat([df_a, df_b, df_c])

,key,num_a,num_b
0,a,1.0,NaN
1,b,2.0,NaN
2,c,3.0,NaN
3,d,4.0,NaN
4,e,5.0,NaN
0,a,NaN,11.0
1,b,NaN,15.0
2,d,NaN,35.0
3,f,NaN,45.0
4,g,NaN,55.0


In [4]:
# axis = 1인 경우 컬럼 역시 재정렬이나 병합 없이 이어붙여짐
pd.concat([df_a, df_b, df_c], axis = 1)

,key,num_a,key,num_b,key,num_a
0,a,1,a,11,f,6
1,b,2,b,15,g,7
2,c,3,d,35,h,8
3,d,4,f,45,i,9
4,e,5,g,55,j,0


### 4-2-2.merge() 함수
- dataframe1.merge(dataframe2) = dataframe1과 dataframe2를 병합
- how 인자를 통해 병합방식을 설정 가능
 - inner join: 머지 대상의 공통된 키만을 추려내 반환
 - left join: "왼쪽" df의 모든 키와 "오른쪽" df의 공통된 키를 반환
 - right join: "오른쪽" df의 모든 키와 "왼쪽" df의 공통된 키를 반환
 - outer join: 머지 대상의 모든 키를 na 포함하여 반환
- on 인자를 통해 병합의 기준이 되는 컬럼을 설정 가능:<br>미설정시 이름이 같은 컬럼들이 기준이 됨

In [5]:
# on 인자를 생략하여, 이름이 같은 "key" 컬럼이 기준이 됨

df_a.merge(df_b, how = 'outer')

,key,num_a,num_b
0,a,1.0,11.0
1,b,2.0,15.0
2,c,3.0,NaN
3,d,4.0,35.0
4,e,5.0,NaN
5,f,NaN,45.0
6,g,NaN,55.0


In [6]:
# 이름이 같은 컬럼이 복수일 경우, 일부만 키 컬럼으로 설정했을 때와 비교

df_a.merge(df_c, how = 'outer') # num_a 컬럼 역시 키 컬럼이며 병합의 대상이 아님

,key,num_a
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
5,f,6
6,g,7
7,h,8
8,i,9
9,j,0


In [7]:
df_a.merge(df_c, on = 'key', how = 'outer') # num_a 컬럼 역시 병합의 대상이 됨

,key,num_a_x,num_a_y
0,a,1.0,NaN
1,b,2.0,NaN
2,c,3.0,NaN
3,d,4.0,NaN
4,e,5.0,NaN
5,f,NaN,6.0
6,g,NaN,7.0
7,h,NaN,8.0
8,i,NaN,9.0
9,j,NaN,0.0


In [8]:
# "왼쪽"과 "오른쪽"의 df에 공통된 컬럼명이 없을 경우:
# 실습을 위해 df_b의 key 컬럼을 id로 변경한 df_d를 생성

df_d = df_b.rename(columns = {'key': 'id'})

In [9]:
# left_on과 right_on을 각각 지정

df_a.merge(df_d, left_on = "key", right_on = "id", how = "outer")

,key,num_a,id,num_b
0,a,1.0,a,11.0
1,b,2.0,b,15.0
2,c,3.0,NaN,NaN
3,d,4.0,d,35.0
4,e,5.0,NaN,NaN
5,NaN,NaN,f,45.0
6,NaN,NaN,g,55.0


### 4-2-3.join() 함수
- 구문은 merge와 비슷하나 결과는 concat과 비슷
 - 동일한 컬럼명이 있을 경우 에러 발생: 에러 처리 위해 lsuffix, rsuffix 지정 가능
 - 동일한 컬럼명의 내용을 기준으로 병합하고 싶다면 df.set_index('컬럼명')로 처리 필요

In [10]:
# "왼쪽"과 "오른쪽" df에 각각 접미사를 붙여 조인

df_a.join(df_b, lsuffix = "_a", rsuffix = "_b") # how 인자 설정 가능

,key_a,num_a,key_b,num_b
0,a,1,a,11
1,b,2,b,15
2,c,3,d,35
3,d,4,f,45
4,e,5,g,55


In [11]:
# 공통된 컬럼명을 가진 key를 인덱스로 처리하여 조인

df_a_join = df_a.set_index('key')
df_b_join = df_b.set_index('key')

df_a_join.join(df_b_join) # how 인자 미설정시 left join이 기본값

,num_a,num_b
key,,
a,1,11.0
b,2,15.0
c,3,NaN
d,4,35.0
e,5,NaN


### 4-2-4. 연습 데이터를 활용한 예제

In [12]:
#연습용 데이터 호출

salary_1 = pd.read_csv('~/data/salary_1.csv')
salary_2 = pd.read_csv('~/data/salary_2.csv')

In [13]:
# 데이터 확인

salary_1

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
3994,44.0,Male,3,Data Scientist,15.0,150000,USA,African American,0
3995,28.0,Male,1,Software Developer,3.0,60000,China,Korean,0
3996,31.0,Male,1,Software Engineer,5.0,80000,UK,Mixed,0
3997,27.0,Female,2,Marketing Manager,3.0,60000,UK,White,0


In [14]:
salary_2

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,29,Female,1,Human Resources Coordinator,2,45000,China,Chinese,0
1,23,Male,0,Sales Associate,1,25000,USA,Asian,0
2,32,Female,2,Software Developer,7,90000,USA,African American,0
3,44,Male,1,Operations Manager,18,170000,USA,Hispanic,0
4,26,Female,1,Marketing Coordinator,2,40000,Canada,Asian,0
...,...,...,...,...,...,...,...,...,...
2680,49,Female,3,Director of Marketing,20,200000,UK,Mixed,0
2681,32,Male,0,Sales Associate,3,50000,Australia,Australian,0
2682,30,Female,1,Financial Manager,4,55000,China,Chinese,0
2683,46,Male,2,Marketing Manager,14,140000,China,Korean,0


In [15]:
# 두 데이터프레임 병합

salary_df = pd.concat([salary_1, salary_2])
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
2680,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0
2681,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0
2682,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0
2683,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0


In [16]:
# 인덱스가 중복되어 있음에 주의

salary_df.loc[0]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
0,29.0,Female,1,Human Resources Coordinator,2.0,45000,China,Chinese,0


In [17]:
# 인덱스를 재설정하는 방법 1

salary_df.reset_index(inplace = True) # index를 재설정하고 기존 index는 컬럼으로 옮김
salary_df # index 컬럼이 새로 생겼음에 주의: drop = True로 해결할 수 있음

,index,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,4,52.0,Male,2,Director,20.0,200000,USA,Asian,0
...,...,...,...,...,...,...,...,...,...,...
6679,2680,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0
6680,2681,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0
6681,2682,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0
6682,2683,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0


In [18]:
salary_df.drop('index', axis = 1, inplace = True) # 또는 수동으로 'index' 컬럼 제거
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0


In [19]:
# salary 컬럼을 국가별 소비자물가지수로 환산하기 위해 데이터 호출

cpi = pd.read_csv('~/data/cpi.csv')
cpi

,Country,Reference,Last,Previous,Units,Frequency
0,United States,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
1,China,Sep-23,100,100.1,"Index CPPY=100, NSA",Monthly
2,Germany,Sep-23,117.9,117.5,"Index 2020=100, CDASA",Monthly
3,France,Sep-23,118.52,118.2,"Index 2015=100, SA",Monthly
4,United Kingdom,Sep-23,132,131.3,"Index 2015=100, NSA",Monthly
5,Brazil,Sep-23,"6,700","6,683","Index Dec1993=100, NSA",Monthly
6,Italy,Aug-23,120.1,119.7,"Index 2015=100, NSA",Monthly
7,Russian Federation,Nov-21,205.5,203.5,"Index 2010=100, NSA",Monthly
8,Canada,Sep-23,158.7,158.4,"Index 2002=100, SA",Monthly
9,Australia,2023 Q3,135.3,133.7,"Index FY 2012=100, NSA",Quarterly


In [20]:
# merge의 기준이 될 수 있는 country 값이 각 df에서 다르므로, 전처리를 위해 확인

print(f"salary_df: {salary_df['Country'].unique()}")
print(f"cpi: {cpi['Country'].unique()}")

salary_df: ['UK' 'USA' 'Canada' 'China' 'Australia']
cpi: ['United States' 'China' 'Germany' 'France' 'United Kingdom' 'Brazil'
 'Italy' 'Russian Federation' 'Canada' 'Australia' 'Spain' 'South Korea'
 'Mexico' 'Indonesia' 'Netherlands']


In [21]:
# 병합대상인 cpi의 컬럼값을 확인된 결과에 따라 전처리

cpi['Country'].replace({'United States': 'USA', 'United Kingdom': 'UK'}, inplace = True)

In [22]:
# 'Country' 컬럼을 기준으로 left join

salary_df = salary_df.merge(cpi, on = 'Country', how = 'left')
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,Reference,Last,Previous,Units,Frequency
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,Sep-23,132,131.3,"Index 2015=100, NSA",Monthly
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,Sep-23,158.7,158.4,"Index 2002=100, SA",Monthly
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0,Sep-23,132,131.3,"Index 2015=100, NSA",Monthly
6680,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0,2023 Q3,135.3,133.7,"Index FY 2012=100, NSA",Quarterly
6681,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0,Sep-23,100,100.1,"Index CPPY=100, NSA",Monthly
6682,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0,Sep-23,100,100.1,"Index CPPY=100, NSA",Monthly


In [24]:
# 분석과 무관한 컬럼 삭제

salary_df.drop(['Reference', 'Previous', 'Units', 'Frequency'], axis = 1, inplace = True)
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,Last
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,132
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,307.48
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,158.7
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,307.48
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,307.48
...,...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0,132
6680,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0,135.3
6681,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0,100
6682,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0,100


In [ ]:
# 'Last' 컬럼명을 변경

salary_df.rename('Last': 'CIP')